# INTRO TO PYTHON MOOC CHALLENGE
## Latifah Mojisola Salaudeen


First, I'll like to open the binary file and check for images.

In [1]:
# Defining image markers and trailers
# PNG
pngMarker = b'\x89\x50\x4E\x47\x0D\x0A\x1A\x0A'
pngTrailer = b'\x49\x45\x4E\x44\xAE\x42\x60\x82'

# GIF
gifMarker1 = b'\x47\x49\x46\x38\x37\x61'
gifMarker2 = b'\x47\x49\x46\x38\x39\x61'
gifTrailer =  b'\x00\x3B'

## JPEG
jpegMarker = b'\xFF\xD8\xFF'
jpegTrailer = b'\xFF\xD9'

## JBIG2
jbig2Marker = b'\x97\x4A\x42\x32\x0D\x0A\x1A\x0A'
jbig2Trailer = b'\x03\x33\x00\x01\x00\x00\x00\x00'

In [2]:
# open the file and check for the markers
#wrapping this in a try block to catch errors
try:
    with open("dstl_MOOC_Challenge_v1.bin", "rb") as file:
        readFile = file.read()
        if pngMarker in readFile:
            print("PNG in file")
        elif jpegMarker in readFile:
            print("jpeg in file")
        elif gifMarker1 in readFile or gifMarker2 in readFile:
            print("GIF in file")
        elif jbig2Marker in readFile:
            print("JBIG2 in file")
        else: print("No images in file")
except FileNotFoundError: 
        print("File not found.")


jpeg in file


Since JPEG is in file, I'd like to get the markers, trailers, and the data in between those.
First, I'll get the indexes of the markers and trailers in the binary document.

In [3]:
#Using regex, find the positions(index) of all the markers and trailer
import re

#A function to find Jpeg signatures in a file and return its markers and trailers
def findJpegSignatures(filePath):
    try:
        with open(filePath, "rb") as file:
            readFile = file.read()

            # defining empty lists to hold markers and trailers indices
            markersIndex = []
            trailersIndex = []

            #finditer() creates an iterable object
            markersSpan = re.finditer(jpegMarker, readFile)
            trailersSpan = re.finditer(jpegTrailer, readFile)

            for i in markersSpan:
                markersIndex.append((i.start()))

            for i in trailersSpan:
                trailersIndex.append((i.start()))

            return markersIndex, trailersIndex
        
    except FileNotFoundError: 
        print("File not found.")

    
#get data between marker and trailer
dstlFile = "dstl_MOOC_Challenge_v1.bin"

markersIndex, trailersIndex = findJpegSignatures(dstlFile)

with open(dstlFile, "rb") as file:
    readDstlFile = file.read()

for i in range(len(markersIndex)): #loop runs for the number of items in markersIndex
    outputFilename = f'{"extracted-images/pic"}{i+1}.jpeg'
    with open(outputFilename, 'wb') as jpegFile:
        jpegFile.write(readFile[markersIndex[i]:trailersIndex[i+1]])
    
    print(f"JPEG extracted and saved as '{outputFilename}'")

JPEG extracted and saved as 'extracted-images/pic1.jpeg'
JPEG extracted and saved as 'extracted-images/pic2.jpeg'
JPEG extracted and saved as 'extracted-images/pic3.jpeg'
JPEG extracted and saved as 'extracted-images/pic4.jpeg'
JPEG extracted and saved as 'extracted-images/pic5.jpeg'
JPEG extracted and saved as 'extracted-images/pic6.jpeg'
JPEG extracted and saved as 'extracted-images/pic7.jpeg'
JPEG extracted and saved as 'extracted-images/pic8.jpeg'
JPEG extracted and saved as 'extracted-images/pic9.jpeg'
JPEG extracted and saved as 'extracted-images/pic10.jpeg'
JPEG extracted and saved as 'extracted-images/pic11.jpeg'
JPEG extracted and saved as 'extracted-images/pic12.jpeg'
JPEG extracted and saved as 'extracted-images/pic13.jpeg'
JPEG extracted and saved as 'extracted-images/pic14.jpeg'
JPEG extracted and saved as 'extracted-images/pic15.jpeg'
JPEG extracted and saved as 'extracted-images/pic16.jpeg'
JPEG extracted and saved as 'extracted-images/pic17.jpeg'
JPEG extracted and save

Now, I want to extract the rest of the data minus the jpeg. I can move from index 0 to the first marker; then move from trailer to marker (from trailer 1 to marker 2)

In [4]:
markersIndex, trailersIndex = findJpegSignatures(dstlFile)

with open(dstlFile, "rb") as file:
    readDstlFile = file.read()


#empty string to hold the bin data
nonJpegData = b""

#from beginning of the file to the first marker
nonJpegData += readDstlFile[0:markersIndex[0]] 

#can't do marker 1(index 0) cos there's no trailer before
#loop runs for the number of items-1 in markersIndex (89th marker is in index 88)
for i in range(len(markersIndex)-1): 
    nonJpegData += readDstlFile[trailersIndex[i]:markersIndex[i+1]] #first iteration = trailer1 to marker 2

#append trailer 89 to the end
nonJpegData += readDstlFile[trailersIndex[len(markersIndex)]:]

with open("nonJpegData.bin", 'wb') as output_file:
    output_file.write(nonJpegData)

    print(f'Non-JPEG data extracted and saved to {"nonJpegData"}')

Non-JPEG data extracted and saved to nonJpegData


With Non-JPEG data extracted. I'd like to see what that looks like as txt.

In [55]:
# A function to convert a binary file to a text file
def binaryToText(inputFile, outputFile, encoding='utf-8', errors="ignore"):
    try:
        # Read input file
        with open(inputFile, 'rb') as file:
            binaryData = file.read()
        
        # Decode
        textFromBinary = binaryData.decode(encoding, errors)

        # Write decoded text to file
        with open(outputFile, 'w') as file:
            file.write(textFromBinary)
        
        print(f'Decoded text saved to {outputFile}')
    
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except UnicodeDecodeError as e:
        print(f"Error decoding binary data: {e}")

#converting nonJpegData to text
inputFile = 'nonJpegData.bin'  
outputFile = 'nonJpegDataFromBinary-utf8.txt'

binaryToText(inputFile, outputFile)


Decoded text saved to nonJpegDataFromBinary-utf8.txt


In [5]:
#finding what's between the pictures
# what is the length of the characters between the images
markersIndex, trailersIndex = findJpegSignatures(dstlFile)

for i in range(len(markersIndex)-1):
    print(trailersIndex[i]-markersIndex[i+1])

print(len(readFile)-trailersIndex[89])

#it seems there are 241 characters between the jpeg images and 225 from the end of the trailer to the end of the file




-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
-241
225


In [49]:

beforeFirstImage = readFile[:markersIndex[0]]
beforeFirstImageText = beforeFirstImage.decode(encoding="ascii", errors="ignore")
print("beforeFirstImage as hex: ",beforeFirstImage)
print("beforeFirstImage as txt: "+beforeFirstImageText)
#the first characters before the first image seems to contain null data

beforeFirstImage as hex:  b'\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa'
beforeFirstImage as txt: 


Trying to see the last 225 indices (bytes) of the data

In [6]:
last225bytes = readFile[-225:]
print(last225bytes)
decodedLast225Bytes = last225bytes.decode(encoding="ascii", errors="ignore")
print(decodedLast225Bytes)

b'\xff\xd9\xff\xbb\xff\xbb\xff\xbb\xff\xbb\xe5\x86LB\x00<\xda\xbf\xd2\x00\x00\x00\x95\x0f\x00\x00=\n\xf6B\x00\x00\x00@2023-10-08 19:30:00\x00\x00LB\x00\x00\xe0@\xbc\xf7XB\x00\x00\x80\xbf\x00\x00(BO\xbc\x8eA(\x03MB\x00\x00LB\x00\x00pAn\xa31A\x9b\x8e\xd8\xbf\x00\x00\x80\xbf\x00\x00$BmV\xf5A\x00\x00\xfaC\x00{\x92LB\x00\x00LB\x00\x00\x00A\xc6\xdc\x0bBy\xea\xed\xbf\x00\x00\x80\xbf\x00\x00LB\x12\x14\xfbA\x00\x80;E\x00Z\x86LB\x00\x00LB\x00\x00\xe0@\x82GWBo\x11\xda\xbf\x00\x00\x80\xbf\x00\x00(B\xfc\xb9\x91A\x00\x00zE\x01\x97\x1bMB\x00\x00LB\x00\x00\x80A]\xfe\x13BI\xf4\xec\xbf\x00\x00\x80\xbf\x00\x00LB\xe5\xd0\x8a@\x00\x00HC\x00'
LB <     =
B   @2023-10-08 19:30:00  LB  @XB    (BOA(MB  LB  pAn1A    $BmVA  C {LB  LB   ABy    LBA ;E ZLB  LB  @GWBo    (BA  zEMB  LB  A]BI    LB@  HC 


Is there a pattern between the 241 bytes of data between the images?

In [7]:
bytesBetweenImages = b''
decodedbytesBetweenImages = ""
for i in range(len(markersIndex)-1):
    print(readFile[trailersIndex[i]:markersIndex[i+1]], "\n")
    bytesBetweenImage = readFile[trailersIndex[i]:markersIndex[i+1]]
    decodedbytesBetweenImage = bytesBetweenImage.decode(encoding="ascii", errors="ignore")+"\n\n"
    decodedbytesBetweenImages+=decodedbytesBetweenImage
    
print(decodedbytesBetweenImages)

# Write decoded text to file
with open("decodedbytesBetweenImages.txt", 'w') as file:
    file.write(decodedbytesBetweenImages)

print("Decoded text saved")

b'\xff\xd9\xff\xbb\xff\xbb\xff\xbb\xff\xbb\xf4]LB\x9f\xf0\xe6\xbf\xd2\x00\x00\x00\x95\x0f\x00\x00\x85\xebAA\x00\x00\x00\x002023-10-08 19:28:31\x00\x00LB\x00\x00\xa0@\x85z\xf2A\x00\x00\x80\xbf\x00\x00@B\xd1\xf7rA(\x03MB\x00\x00LB\x00\x00pAn\xa31A\x9b\x8e\xd8\xbf\x00\x00\x80\xbf\x00\x00$BmV\xf5A\x00\x00\xfaC\x00{\x92LB\x00\x00LB\x00\x00\x00A\xc6\xdc\x0bBy\xea\xed\xbf\x00\x00\x80\xbf\x00\x00LB\x12\x14\xfbA\x00\x80;E\x00Z\x86LB\x00\x00LB\x00\x00\xe0@\x82GWBo\x11\xda\xbf\x00\x00\x80\xbf\x00\x00(B\xfc\xb9\x91A\x00\x00zE\x01\x97\x1bMB\x00\x00LB\x00\x00\x80A]\xfe\x13BI\xf4\xec\xbf\x00\x00\x80\xbf\x00\x00LB\xe5\xd0\x8a@\x00\x00HC\x00\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa\xff\xaa' 

b'\xff\xd9\xff\xbb\xff\xbb\xff\xbb\xff\xbb\xee^LB\xf9\xe5\xe6\xbf\xd2\x00\x00\x00\x95\x0f\x00\x00\x9a\x99MA\x00\x00\x80?2023-10-08 19:28:32\x00\x00LB\x00\x00\xa0@\xb1\xed\x06B\x00\x00\x80\xbf\x00\x00@B\x16@`A(\x03MB\x00\x00LB\x00\x00pAn\xa31A\x9b\x8e\xd8\xbf\x00\x00\x80\xbf\x00\x00$BmV\xf5A\x00\x00\